# Airbnb Price Predictor Project
### Using the power of python and machine learning we will gather information of all listings within the country of the United States of America in order to develop a machine learning model that will help predict the price of the listing

#### information gathered from http://insideairbnb.com/get-the-data.html

### First we import the dependencies

In [1]:
import os
import numpy as np
import pandas as pd
from glob import glob
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile

In [53]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

### Next we navigate to the proper folder within our machine

In [2]:
os.getcwd()

'C:\\Users\\brenden.lemay'

In [3]:
os.chdir('./Downloads/Airbnb Data')

### Now we import our data utilizing a glob object and a list comprehension

In [4]:
filenames = glob('./listings*.csv')
dataframes = [pd.read_csv(f) for f in filenames]


In [5]:
type(dataframes)

list

In [6]:
dataframes[0].head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,38585,Charming Victorian home - twin beds + breakfast,165529,Evelyne,NaN,28804,35.65146,-82.62792,Private room,50,1,138,2020-02-16,1.17,1,9
1,80905,French Chic Loft,427027,Celeste,NaN,28801,35.59779,-82.55540,Entire home/apt,765,1,108,2020-03-15,1.01,11,323
2,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,320564,Lisa,NaN,28801,35.60670,-82.55563,Entire home/apt,75,30,89,2019-11-30,0.83,2,297
3,155305,Cottage! BonPaul + Sharky's Hostel,746673,BonPaul,NaN,28806,35.57864,-82.59578,Entire home/apt,99,1,244,2020-06-22,2.25,7,180
4,156805,"Private Room ""Ader"" at BPS Hostel",746673,BonPaul,NaN,28806,35.57864,-82.59578,Private room,65,1,67,2020-01-01,0.63,7,0


###### Here we can see that we have a list of DataFrames. We need to get these into one DataFrame in order to begin wrangling our data

In [7]:
df = pd.DataFrame()
for frame in dataframes:
    df = df.append(frame)

In [8]:
df.shape

(251743, 16)

In [9]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,38585,Charming Victorian home - twin beds + breakfast,165529,Evelyne,NaN,28804,35.65146,-82.62792,Private room,50,1,138,2020-02-16,1.17,1,9
1,80905,French Chic Loft,427027,Celeste,NaN,28801,35.59779,-82.55540,Entire home/apt,765,1,108,2020-03-15,1.01,11,323
2,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,320564,Lisa,NaN,28801,35.60670,-82.55563,Entire home/apt,75,30,89,2019-11-30,0.83,2,297
3,155305,Cottage! BonPaul + Sharky's Hostel,746673,BonPaul,NaN,28806,35.57864,-82.59578,Entire home/apt,99,1,244,2020-06-22,2.25,7,180
4,156805,"Private Room ""Ader"" at BPS Hostel",746673,BonPaul,NaN,28806,35.57864,-82.59578,Private room,65,1,67,2020-01-01,0.63,7,0


In [10]:
# Need to drop duplicates since I'm pretty sure I donwloaded Austin's data more than once
df.drop_duplicates(inplace=True)

In [11]:
df.shape

(249685, 16)

In [12]:
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [13]:
os.chdir('./GZ Files')

In [14]:
zip_filenames = glob('./listings*.csv.gz')


In [19]:
zip_df = pd.DataFrame()
for file in zip_filenames:
    zip_df = zip_df.append(pd.read_csv(file, compression='infer'))

In [20]:
zip_df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       ...
       'instant_bookable', 'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'],
      dtype='object', length=106)

In [21]:
zip_df.shape

(250720, 106)

In [22]:
# Need to drop duplicates since I'm pretty sure I donwloaded Austin's data more than once
zip_df.drop_duplicates(inplace=True)

In [23]:
# GZ files shape
zip_df.shape

(247280, 106)

In [24]:
# CSV files shape
df.shape

(249685, 16)

In [25]:
common_cols = ['name', 'host_id', 'host_name', 'neighbourhood', 'latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 
               'last_review', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']

In [26]:
# Dropping common columns in order to avoid the _x and _y columns
trimmed = zip_df.drop(columns=common_cols)

In [27]:
trimmed.shape

(247280, 92)

#### Now that we have our two DataFrames, we need to join the two in order to get access to all the columns available

In [30]:
outer = pd.merge(df, trimmed, how='right', on='id')

In [31]:
outer.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       ...
       'license', 'jurisdiction_names', 'instant_bookable',
       'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms'],
      dtype='object', length=107)

In [32]:
outer.shape

(262656, 107)

In [54]:
outer.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,listing_url,scrape_id,last_scraped,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_url,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,is_location_exact,property_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,calendar_last_scraped,number_of_reviews_ltm,first_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,38585,Charming Victorian home - twin beds + breakfast,165529,Evelyne,NaN,28804,35.65146,-82.62792,Private room,50,1,138,2020-02-16,1.17,1,9,https://www.airbnb.com/rooms/38585,20200625032333,2020-06-25,Per the City Council of Asheville. Due to the ...,"Charming room with 2 twin size beds, furnished...",Per the City Council of Asheville. Due to the ...,none,Our North Asheville neighborhood stretches alo...,I can lend you a yoga mat or you can go to our...,"You need a car, but UBER and Lyft are availabl...",Safe parking in our driveway. You also have fu...,"I love entertaining, enjoy chatting with guest...",This list is based on previous experiences: * ...,NaN,NaN,https://a0.muscache.com/im/pictures/216165/755...,NaN,https://www.airbnb.com/users/show/165529,2010-07-13,"Asheville, North Carolina, United States","(Hidden by Airbnb) Likes:\r\nDancing, Horse Ri...",NaN,NaN,93%,t,https://a0.muscache.com/im/users/165529/profil...,https://a0.muscache.com/im/users/165529/profil...,NaN,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'off...",t,f,"Asheville, NC, United States",28804,NaN,Asheville,NC,28804,North Carolina Mountains,"Asheville, NC",US,United States,t,House,2,1.0,1.0,2.0,Real Bed,"{Internet,Wifi,""Air conditioning"",Kitchen,""Fre...",NaN,NaN,NaN,$150.00,$0.00,1,$15.00,3,1,1,3,3,1.0,3.0,6 weeks ago,t,0,0,9,2020-06-25,10,2010-10-28,96.0,10.0,10.0,10.0,10.0,10.0,10.0,f,NaN,"{""NORTH CAROLINA"","" BUNCOMBE""}",f,f,moderate,t,t,0,1,0
1,80905,French Chic Loft,427027,Celeste,NaN,28801,35.59779,-82.55540,Entire home/apt,765,1,108,2020-03-15,1.01,11,323,https://www.airbnb.com/rooms/80905,20200625032333,2020-06-25,Let yourself melt into the delectable décor of...,Have you ever gazed at dreamy photos in a maga...,Let yourself melt into the delectable décor of...,none,"Quiet, tucked away from the hustle and bustle ...",there is Apple TV not cable TV (there is no op...,Park your car and walk anywhere and everywhere...,NaN,I am easily accessible and available if you ne...,THE GUEST WHO BOOKS AGREES TO: • Be present at...,NaN,NaN,https://a0.muscache.com/im/pictures/29807425/6...,NaN,https://www.airbnb.com/users/show/427027,2011-03-07,"Asheville, North Carolina, United States",Locally-owned and operated Asheville S

In [41]:
outer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 262656 entries, 0 to 262655
Columns: 107 entries, id to calculated_host_listings_count_shared_rooms
dtypes: float64(21), int64(22), object(64)
memory usage: 216.4+ MB


In [52]:
# Need a different way to get a count of nulls held in columns
for col in outer:
    print(col, outer[col].isna().sum())

id 0
name 29
host_id 0
host_name 134
neighbourhood_group 144935
neighbourhood 0
latitude 0
longitude 0
room_type 0
price 0
minimum_nights 0
number_of_reviews 0
last_review 53447
reviews_per_month 53447
calculated_host_listings_count 0
availability_365 0
listing_url 0
scrape_id 0
last_scraped 0
summary 9890
space 58797
description 5758
experiences_offered 0
neighborhood_overview 84274
notes 129174
transit 95006
access 109012
interaction 91441
house_rules 83335
thumbnail_url 262656
medium_url 262656
picture_url 0
xl_picture_url 262656
host_url 0
host_since 134
host_location 712
host_about 89886
host_response_time 83821
host_response_rate 83821
host_acceptance_rate 45643
host_is_superhost 134
host_thumbnail_url 134
host_picture_url 134
host_neighbourhood 67149
host_listings_count 134
host_total_listings_count 134
host_verifications 0
host_has_profile_pic 134
host_identity_verified 134
street 0
neighbourhood_cleansed 0
neighbourhood_group_cleansed 144935
city 168
state 73
zipcode 1965
mark

In [37]:
outer.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
count,2.626560e+05,2.626560e+05,262656.000000,262656.000000,262656.000000,2.626560e+05,262656.000000,209209.000000,262656.000000,262656.000000,...,206324.000000,205970.000000,205991.000000,205910.000000,205976.000000,205889.000000,205883.000000,262656.000000,262656.000000,262656.000000
mean,2.494341e+07,8.984199e+07,35.139183,-102.210781,266.284414,3.896152e+02,34.347770,1.458536,14.561145,153.109900,...,95.005554,9.692446,9.513586,9.795338,9.777974,9.720184,9.491196,12.487105,1.424704,0.262644
std,1.285779e+07,9.408719e+07,6.635707,24.791067,795.220788,1.951220e+05,63.076655,1.690128,40.534805,139.454749,...,8.060641,0.789469,0.918584,0.687765,0.722285,0.681619,0.873056,39.134211,6.032677,2.726490
min,1.090000e+02,2.300000e+01,18.920990,-159.714900,0.000000,1.000000e+00,0.000000,0.010000,1.000000,0.000000,...,20.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000
25%,1.501010e+07,1.292539e+07,30.289040,-118.372422,75.000000,1.000000e+00,1.000000,0.250000,1.000000,0.000000,...,94.000000,10.000000,9.000000,10.000000,10.000000,10.000000,9.000000,0.000000,0.000000,0.000000
50%,2.529632e+07,5.030623e+07,36.095400,-97.747770,125.000000,2.000000e+00,8.000000,0.830000,2.000000,128.000000,...,97.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000,0.000000,0.000000
75%,3.673115e+07,1.439690e+08,40.710853,-77.034300,220.000000,5.000000e+00,39.000000,2.120000,6.000000,302.000000,...,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,3.000000,1.000000,0.000000
max,4.431889e+07,3.569385e+08,47.734620,-70.995950,25000.000000,1.000000e+08,951.000000,50.290000,393.000000,365.000000,...,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,393.000000,148.000000,67.000000


In [38]:
#Calculate the correlation coefficients
corr = outer.corr()
#plot it in the next line
corr.round(2).style.background_gradient(cmap='coolwarm')

C:\Users\brenden.lemay\Anaconda3\lib\site-packages\pandas\io\formats\style.py:1089: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(s.to_numpy()) if vmin is None else vmin
C:\Users\brenden.lemay\Anaconda3\lib\site-packages\pandas\io\formats\style.py:1090: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(s.to_numpy()) if vmax is None else vmax


##### create lists of dataframe columns and plot the information in the df[lists]